# 학습 및 검증 실시

- SSD 학습과 검증의 실시를 수행합니다. 로컬 머신으로 동작을 확인한 뒤, AWS의 GPU 머신으로 계산합니다.





1.	SSD 학습을 구현한 내용입니다

# 사전 준비

- AWS EC2의 GPU 인스턴스를 사용합니다
- "utils" 폴더의 ssd_model.py를 실행합니다

In [9]:
# 패키지 import
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [10]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

사용 중인 장치: cuda:0


# Dataset과 DataLoader를 작성한다

In [12]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# 파일 경로 리스트를 취득
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Dataset 작성
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR) 색의 평균값
input_size = 300  # 이미지의 input 크기를 300×300으로 설정

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader를 작성
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 사전 오브젝트로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 네트워크 모델을 작성한다

In [13]:
from utils.ssd_model import SSD

# SSD300 설정
ssd_cfg = {
    'num_classes': 21,  # 배경 클래스를 포함한 총 클래스 수
    'input_size': 300,  # 이미지의 입력 크기
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 출력할 DBox의 화면비의 종류
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 각 source의 이미지 크기
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOX의 크기(최소)
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOX의 크기(최대)
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD 네트워크 모델
net = SSD(phase="train", cfg=ssd_cfg)

# SSD의 초기 가중치를 설정
# ssd의 vgg 부분에 가중치를 로드한다
vgg_weights = torch.load('weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssd의 기타 네트워크의 가중치는 He의 초기치로 초기화
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # 바이어스 항이 있는 경우
            nn.init.constant_(m.bias, 0.0)


# He의 초기치를 적용
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPU를 사용할 수 있는지 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다')


사용 중인 장치: cuda:0
네트워크 설정 완료: 학습된 가중치를 로드했습니다


# 손실함수 및 최적화 기법의 설정

In [14]:
from utils.ssd_model import MultiBoxLoss

# 손실함수의 설정
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 최적화 기법의 설정
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 학습 및 검증을 실시

In [15]:
import tqdm

# 모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 중인 장치:", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 반복자의 카운터 설정
    iteration = 1
    epoch_train_loss = 0.0  # epoch의 손실합
    epoch_val_loss = 0.0  # epoch의 손실합
    logs = []

    # epoch 루프
    for epoch in range(num_epochs+1):

        # 시작 시간을 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증을 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련모드로
                print('(train)')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # 모델을 검증모드로
                    print('-------------')
                    print('(val)')
                else:
                    # 검증은 10번에 1번만 실시
                    continue

            # 데이터 로더에서 minibatch씩 꺼내 루프
            iterator = tqdm.tqdm(dataloaders_dict[phase])
            #for images, targets in dataloaders_dict[phase]:
            for images, targets in iterator:
                # GPU를 사용할 수 있으면, GPU에 데이터를 보낸다
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # 리스트의 각 요소의 텐서를 GPU로

                # optimizer를 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    # 순전파(forward) 계산
                    outputs = net(images)

                    # 손실 계산
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 훈련시에는 역전파(Backpropagation)
                    if phase == 'train':
                        loss.backward()  # 경사 계산

                        # 경사가 너무 커지면 계산이 불안정해지므로, clip에서 최대라도 경사 2.0에 고정
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # 파라미터 갱신

                        #if (iteration % 10 == 0):  # 10iter에 한 번, loss를 표시
                        t_iter_finish = time.time()
                        duration = t_iter_finish - t_iter_start
                        iterator.set_description('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                        iteration, loss.item(), duration))
                        t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 검증시
                    else:
                        epoch_val_loss += loss.item()

        # epoch의 phase 당 loss와 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # 로그를 저장
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        # 네트워크를 저장한다
        if ((epoch+1) % 50 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [16]:
# 학습 및 검증 실시
num_epochs= 200 
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 중인 장치: cuda:0
-------------
Epoch 1/50
-------------
(train)


반복 715 || Loss: 9.1572 || 10iter: 10.6015 sec.: 100%|██████████| 715/715 [37:26<00:00,  3.14s/it]  


-------------
epoch 1 || Epoch_TRAIN_Loss:6317.5546 ||Epoch_VAL_Loss:0.0000
timer:  2246.1495 sec.
-------------
Epoch 2/50
-------------
(train)


반복 1430 || Loss: 6.0575 || 10iter: 0.2212 sec.: 100%|██████████| 715/715 [40:59<00:00,  3.44s/it]   


-------------
epoch 2 || Epoch_TRAIN_Loss:5042.7311 ||Epoch_VAL_Loss:0.0000
timer:  2459.1323 sec.
-------------
Epoch 3/50
-------------
(train)


...

-------------
(val)


100%|██████████| 728/728 [02:54<00:00,  4.18it/s]


-------------
epoch 50 || Epoch_TRAIN_Loss:2329.5179 ||Epoch_VAL_Loss:2679.0044
timer:  607.9113 sec.
-------------
Epoch 51/50
-------------
(train)


반복 36465 || Loss: 3.5556 || 10iter: 0.2000 sec.: 100%|██████████| 715/715 [07:14<00:00,  1.65it/s]

-------------
epoch 51 || Epoch_TRAIN_Loss:2323.2449 ||Epoch_VAL_Loss:0.0000
timer:  434.0042 sec.


끝